# Company Research 

Conducting company research, or competitive analysis, is a critical part of any business strategy. In this notebook, we will demonstrate how create a team of agents to address the task. While there are many ways to translate a task into an agentic implementation, we will explore a sequential approach where we will create agents corresponding to steps in the research process, giving them tools to perform their tasks.

- **Search Agent**: Search the web for information about a company.
- **Stock Analysis Agent**: Retrieve the company's stock information from a financial data API, compute basic statistics (current price, 52 week high, 52 week low, etc.), and generate a plot of the stock price year to date and save to a file.
- **Report Agent**: Generate a report based on the information collected by the search and stock analysis agents.


First, let us import the necessary modules. 

In [1]:
from autogen_agentchat.agents import CodingAssistantAgent,ToolUseAssistantAgent
from autogen_agentchat.teams.group_chat import RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient 
from autogen_core.components.tools import FunctionTool

## Defining Tools 



In [2]:
#!pip install yfinance matplotlib pytz numpy pandas python-dotenv requests bs4

def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import os
    import requests
    from dotenv import load_dotenv
    from bs4 import BeautifulSoup
    import time

    load_dotenv()

    api_key = os.getenv('GOOGLE_API_KEY')
    search_engine_id = os.getenv('GOOGLE_SEARCH_ENGINE_ID')
    
    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'key': api_key,
        'cx': search_engine_id,
        'q': query,
        'num': num_results
    }

    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get('items', [])
    
    def get_page_content(url):
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text(separator=' ', strip=True)
            words = text.split()
            content = ''
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += ' ' + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item['link'])
        enriched_results.append({
            'title': item['title'],
            'link': item['link'],
            'snippet': item['snippet'],
            'body': body
        })
        time.sleep(1)  # Be respectful to the servers

    return enriched_results


def analyze_stock(ticker: str) -> dict:
    import yfinance as yf
    import matplotlib.pyplot as plt
    from datetime import datetime, timedelta
    import numpy as np
    from pytz import timezone
    import pandas as pd
    import os

    stock = yf.Ticker(ticker)
    
    # Get historical data (1 year of data to ensure we have enough for 200-day MA)
    end_date = datetime.now(timezone('UTC'))
    start_date = end_date - timedelta(days=365)
    hist = stock.history(start=start_date, end=end_date)
    
    # Ensure we have data
    if hist.empty:
        return {"error": "No historical data available for the specified ticker."}
    
    # Compute basic statistics and additional metrics
    current_price = stock.info.get('currentPrice', hist['Close'].iloc[-1])
    year_high = stock.info.get('fiftyTwoWeekHigh', hist['High'].max())
    year_low = stock.info.get('fiftyTwoWeekLow', hist['Low'].min())
    
    # Calculate 50-day and 200-day moving averages
    ma_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
    ma_200 = hist['Close'].rolling(window=200).mean().iloc[-1]
    
    # Calculate YTD price change and percent change
    ytd_start = datetime(end_date.year, 1, 1, tzinfo=timezone('UTC'))
    ytd_data = hist.loc[ytd_start:]
    if not ytd_data.empty:
        price_change = ytd_data['Close'].iloc[-1] - ytd_data['Close'].iloc[0]
        percent_change = (price_change / ytd_data['Close'].iloc[0]) * 100
    else:
        price_change = percent_change = np.nan
    
    # Determine trend
    if pd.notna(ma_50) and pd.notna(ma_200):
        if ma_50 > ma_200:
            trend = "Upward"
        elif ma_50 < ma_200:
            trend = "Downward"
        else:
            trend = "Neutral"
    else:
        trend = "Insufficient data for trend analysis"
    
    # Calculate volatility (standard deviation of daily returns)
    daily_returns = hist['Close'].pct_change().dropna()
    volatility = daily_returns.std() * np.sqrt(252)  # Annualized volatility
    
    # Create result dictionary
    result = {
        "ticker": ticker,
        "current_price": current_price,
        "52_week_high": year_high,
        "52_week_low": year_low,
        "50_day_ma": ma_50,
        "200_day_ma": ma_200,
        "ytd_price_change": price_change,
        "ytd_percent_change": percent_change,
        "trend": trend,
        "volatility": volatility
    }
    
    # Convert numpy types to Python native types for better JSON serialization
    for key, value in result.items():
        if isinstance(value, np.generic):
            result[key] = value.item()
    
    # Generate plot
    plt.figure(figsize=(12, 6))
    plt.plot(hist.index, hist['Close'], label='Close Price')
    plt.plot(hist.index, hist['Close'].rolling(window=50).mean(), label='50-day MA')
    plt.plot(hist.index, hist['Close'].rolling(window=200).mean(), label='200-day MA')
    plt.title(f"{ticker} Stock Price (Past Year)")
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.legend()
    plt.grid(True)
    
    # Save plot to file
    os.makedirs("coding", exist_ok=True)
    plot_file_path = f"coding/{ticker}_stockprice.png"
    plt.savefig(plot_file_path)
    print(f"Plot saved as {plot_file_path}")
    result["plot_file_path"] = plot_file_path
    
    return result

In [3]:
google_search_tool = FunctionTool(google_search, description="Search Google for information, returns results with a snippet and body content")
stock_analysis_tool = FunctionTool(analyze_stock, description="Analyze stock data and generate a plot")

In [4]:
search_agent = ToolUseAssistantAgent( name="Google_Search_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    tool_schema=[google_search_tool.schema],
    description="Search Google for information, returns top 2 results with a snippet and body content"
)

stock_analysis_agent = ToolUseAssistantAgent( name="Stock_Analysis_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    tool_schema=[stock_analysis_tool.schema],
    description="Analyze stock data and generate a plot"
) 

report_agent = CodingAssistantAgent( name="Report_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a report based on the search and stock analysis results",
    system_message="You are a helpful assistant that can generate a comprehensive report on a given topic based on search and stock analysis results provided. When you done with generating the report, reply with TERMINATE."
)

team = RoundRobinGroupChat([search_agent, stock_analysis_agent, report_agent], tools=[google_search_tool, stock_analysis_tool])

In [5]:
result = await team.run("Write a financial report on American airlines")

print(result)

--------------------------------------------------------------------------------
user:
Write a financial report on American airlines
--------------------------------------------------------------------------------
Google_Search_Agent:
[FunctionCall(id='call_GNyLKL656n3mCbX5SDfqdOyk', arguments='{"query":"American Airlines financial report 2023","num_results":3}', name='google_search')]
--------------------------------------------------------------------------------
tool_agent:
[FunctionExecutionResult(content="[{'title': 'American Airlines reports fourth-quarter and full-year 2023 financial ...', 'link': 'https://news.aa.com/news/news-details/2024/American-Airlines-reports-fourth-quarter-and-full-year-2023-financial-results-CORP-FI-01/default.aspx', 'snippet': 'Jan 25, 2024 ... American Airlines Group Inc. (NASDAQ: AAL) today reported its fourth-quarter and full-year 2023 financial results, including: Record\\xa0...', 'body': 'Just a moment... Enable JavaScript and cookies to continue'